In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lish-moa/train_targets_scored.csv
/kaggle/input/lish-moa/train_drug.csv
/kaggle/input/lish-moa/train_targets_nonscored.csv
/kaggle/input/lish-moa/train_features.csv
/kaggle/input/lish-moa/sample_submission.csv
/kaggle/input/lish-moa/test_features.csv
/kaggle/input/rank-gauss/rankGaussTrafo.py
/kaggle/input/rank-gauss/gauss_rank_scaler.py
/kaggle/input/rank-gauss/rgn.py
/kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.1-py3-none-any.whl
/kaggle/input/pytorchtabnet/pytorch_tabnet-1.2.0-py3-none-any.whl
/kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl
/kaggle/input/iterative-stratification/iterative-stratification-master/setup.py
/kaggle/input/iterative-stratification/iterative-stratification-master/.travis.yml
/kaggle/input/iterative-stratification/iterative-stratification-master/setup.cfg
/kaggle/input/iterative-stratification/iterative-stratification-master/.gitignore
/kaggle/input/iterative-stratification/iterative-stratification-master/LICENSE
/kaggle/inp

In [2]:
!pip install --no-index --find-links /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl pytorch-tabnet
# Iterative Stratification
!pip install /kaggle/input/iterative-stratification/iterative-stratification-master/


Looking in links: /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl
Processing /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl
Processing /kaggle/input/iterative-stratification/iterative-stratification-master
  Created wheel for iterative-stratification: filename=iterative_stratification-0.1.6-py3-none-any.whl size=8401 sha256=7c876c29cae1c314201b5e7fc28bca9567ea80cfc788be2b8a055c89f5a16f11
  Stored in directory: /root/.cache/pip/wheels/b8/47/3f/eb4af42d124f37d23d6f13a4c8bbc32c1d70140e6e1cecb4aa
Successfully built iterative-stratification


In [3]:
### General ###
import os
import sys
import copy
import tqdm
import pickle
import random
import warnings
warnings.filterwarnings("ignore")
sys.path.append("../input/rank-gauss")
os.environ["CUDA_LAUNCH_BLOCKING"] = '1'

### Data Wrangling ###
import numpy as np
import pandas as pd
from scipy import stats
from gauss_rank_scaler import GaussRankScaler

### Data Visualization ###
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")

### Machine Learning ###
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.preprocessing import QuantileTransformer
from sklearn.feature_selection import VarianceThreshold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

### Deep Learning ###
import torch
from torch import nn
import torch.optim as optim
from torch.nn import functional as F
from torch.nn.modules.loss import _WeightedLoss
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
# Tabnet 
from pytorch_tabnet.metrics import Metric
from pytorch_tabnet.tab_model import TabNetRegressor

### Make prettier the prints ###
from colorama import Fore
c_ = Fore.CYAN
m_ = Fore.MAGENTA
r_ = Fore.RED
b_ = Fore.BLUE
y_ = Fore.YELLOW
g_ = Fore.GREEN



In [4]:
seed = 5

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
set_seed(seed)

In [5]:
# Parameters
data_path = "../input/lish-moa/"
no_ctl = True
scale = "rankgauss"
variance_threshould = 0.7
decompo = "PCA"
ncompo_genes = 80
ncompo_cells = 10
encoding = "dummy"
NB_SPLITS = 7 # 7

In [6]:
train = pd.read_csv(data_path + "train_features.csv")
#train.drop(columns = ["sig_id"], inplace = True)

targets = pd.read_csv(data_path + "train_targets_scored.csv")
#train_targets_scored.drop(columns = ["sig_id"], inplace = True)

#train_targets_nonscored = pd.read_csv(data_path + "train_targets_nonscored.csv")

test = pd.read_csv(data_path + "test_features.csv")
#test.drop(columns = ["sig_id"], inplace = True)

submission = pd.read_csv(data_path + "sample_submission.csv")

In [7]:
if no_ctl:
    # cp_type == ctl_vehicle
    print(b_, "not_ctl")
    train = train[train["cp_type"] != "ctl_vehicle"]
    test = test[test["cp_type"] != "ctl_vehicle"]
    targets = targets.iloc[train.index]
    train.reset_index(drop = True, inplace = True)
    test.reset_index(drop = True, inplace = True)
    targets.reset_index(drop = True, inplace = True)
    

 not_ctl


In [8]:
def distributions(num, graphs, items, features, gorc):
    """
    Plot the distributions of gene expression or cell viability data
    """
    for i in range(0, num - 1, 7):
        if i >= 3:
            break
        idxs = list(np.array([0, 1, 2, 3, 4, 5, 6]) + i)
    
        fig, axs = plt.subplots(1, 7, sharey = True)
        for k, item in enumerate(idxs):
            if item >= items:
                break
            graph = sns.distplot(train[features].values[:, item], ax = axs[k])
            graph.set_title(f"{gorc}-{item}")
            graphs.append(graph)

In [9]:
GENES = [col for col in train.columns if col.startswith("g-")]
CELLS = [col for col in train.columns if col.startswith("c-")]

In [10]:
#rank gauss
data_all = pd.concat([train, test], ignore_index = True)
print('data_all.shape',data_all.shape)
cols_numeric = [feat for feat in list(data_all.columns) if feat not in ["sig_id", "cp_type", "cp_time", "cp_dose"]]

print(len(cols_numeric))

mask = (data_all[cols_numeric].var() >= variance_threshould).values

print('mask.shape',mask.shape)
tmp = data_all[cols_numeric].loc[:, mask]
data_all = pd.concat([data_all[["sig_id", "cp_type", "cp_time", "cp_dose"]], tmp], axis = 1)
cols_numeric = [feat for feat in list(data_all.columns) if feat not in ["sig_id", "cp_type", "cp_time", "cp_dose"]]

data_all.shape (25572, 876)
872
mask.shape (872,)


In [11]:
def scale_minmax(col):
    return (col - col.min()) / (col.max() - col.min())

def scale_norm(col):
    return (col - col.mean()) / col.std()

if scale == "boxcox":
    print(b_, "boxcox")
    data_all[cols_numeric] = data_all[cols_numeric].apply(scale_minmax, axis = 0)
    trans = []
    for feat in cols_numeric:
        trans_var, lambda_var = stats.boxcox(data_all[feat].dropna() + 1)
        trans.append(scale_minmax(trans_var))
    data_all[cols_numeric] = np.asarray(trans).T
    
elif scale == "norm":
    print(b_, "norm")
    data_all[cols_numeric] = data_all[cols_numeric].apply(scale_norm, axis = 0)
    
elif scale == "minmax":
    print(b_, "minmax")
    data_all[cols_numeric] = data_all[cols_numeric].apply(scale_minmax, axis = 0)
elif scale == "rankgauss":
    ### Rank Gauss ###
    print(b_, "Rank Gauss")
    scaler = GaussRankScaler()
    data_all[cols_numeric] = scaler.fit_transform(data_all[cols_numeric])
    
else:
    pass


 Rank Gauss


In [12]:
# PCA
if decompo == "PCA":
    print(b_, "PCA")
    GENES = [col for col in data_all.columns if col.startswith("g-")]
    CELLS = [col for col in data_all.columns if col.startswith("c-")]
    
    pca_genes = PCA(n_components = ncompo_genes,
                    random_state = seed).fit_transform(data_all[GENES])
    pca_cells = PCA(n_components = ncompo_cells,
                    random_state = seed).fit_transform(data_all[CELLS])
    
    pca_genes = pd.DataFrame(pca_genes, columns = [f"pca_g-{i}" for i in range(ncompo_genes)])
    pca_cells = pd.DataFrame(pca_cells, columns = [f"pca_c-{i}" for i in range(ncompo_cells)])
    data_all = pd.concat([data_all, pca_genes, pca_cells], axis = 1)
else:
    pass

 PCA


In [13]:
for feat in ["cp_time", "cp_dose"]:
        data_all[feat] = LabelEncoder().fit_transform(data_all[feat])

In [14]:
# Encoding
if encoding == "lb":
    print(b_, "Label Encoding")
    for feat in ["cp_time", "cp_dose"]:
        data_all[feat] = LabelEncoder().fit_transform(data_all[feat])
elif encoding == "dummy":
    print(b_, "One-Hot")
    data_all = pd.get_dummies(data_all, columns = ["cp_time", "cp_dose"])

 One-Hot


In [15]:
GENES = [col for col in data_all.columns if col.startswith("g-")]
CELLS = [col for col in data_all.columns if col.startswith("c-")]

for stats in tqdm.tqdm(["sum", "mean", "std", "kurt", "skew"]):
    data_all["g_" + stats] = getattr(data_all[GENES], stats)(axis = 1)
    data_all["c_" + stats] = getattr(data_all[CELLS], stats)(axis = 1)    
    data_all["gc_" + stats] = getattr(data_all[GENES + CELLS], stats)(axis = 1)

100%|██████████| 5/5 [00:04<00:00,  1.06it/s]


In [16]:
#新训练过程，将每种药物在每个fold中平均分配

# LOAD FILES
#scored = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
drug = pd.read_csv('/kaggle/input/lish-moa/train_drug.csv')
targets_inhibitor = targets.columns[1:]
targets = targets.merge(drug, on='sig_id', how='left') 

# LOCATE DRUGS
vc = targets.drug_id.value_counts()
vc1 = vc.loc[vc<=18].index.sort_values()
vc2 = vc.loc[vc>18].index.sort_values()

# STRATIFY DRUGS 18X OR LESS
dct1 = {}; dct2 = {}
skf = MultilabelStratifiedKFold(n_splits=NB_SPLITS, shuffle=True, 
          random_state=seed)
tmp = targets.groupby('drug_id')[targets_inhibitor].mean().loc[vc1]
for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets_inhibitor])):
    dd = {k:fold for k in tmp.index[idxV].values}
    dct1.update(dd)

# STRATIFY DRUGS MORE THAN 18X
skf = MultilabelStratifiedKFold(n_splits=NB_SPLITS, shuffle=True, 
          random_state=seed)
tmp = targets.loc[targets.drug_id.isin(vc2)].reset_index(drop=True)
for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets_inhibitor])):
    dd = {k:fold for k in tmp.sig_id[idxV].values}
    dct2.update(dd)

# ASSIGN FOLDS
targets['fold'] = targets.drug_id.map(dct1)
targets.loc[targets.fold.isna(),'fold'] =\
    targets.loc[targets.fold.isna(),'sig_id'].map(dct2)
targets.fold = targets.fold.astype('int8')

In [17]:
with open("data_all.pickle", "wb") as f:
    pickle.dump(data_all, f)
with open("data_all.pickle", "rb") as f:
    data_all = pickle.load(f)
# train_df and test_df
features_to_drop = ["sig_id", "cp_type"]
data_all.drop(features_to_drop, axis = 1, inplace = True)
try:
    targets.drop(["sig_id",'drug_id'], axis = 1, inplace = True)
except:
    pass
train_df = data_all[: train.shape[0]]
train_df.reset_index(drop = True, inplace = True)
# The following line it's a bad practice in my opinion, targets on train set
#train_df = pd.concat([train_df, targets], axis = 1)
test_df = data_all[train_df.shape[0]: ]
test_df.reset_index(drop = True, inplace = True)


In [18]:
print(f"{b_}train_df.shape: {r_}{train_df.shape}")
print(f"{b_}test_df.shape: {r_}{test_df.shape}")

X_test = test_df.values
print(f"{b_}X_test.shape: {r_}{X_test.shape}")

train_df.shape: (21948, 947)
test_df.shape: (3624, 947)
X_test.shape: (3624, 947)


In [19]:
MAX_EPOCH = 200
# n_d and n_a are different from the original work, 32 instead of 24
# This is the first change in the code from the original
tabnet_params = dict(
    n_d = 24,
    n_a = 40,
    n_steps = 1,
    gamma = 1.3,
    lambda_sparse = 0,
    optimizer_fn = optim.Adam,
    #optimizer_fn = optim.SGD,
    optimizer_params = dict(lr = 2e-2, weight_decay = 1e-5),
    #optimizer_params = dict(lr = 6e-2,momentum=0.9,
    #                  weight_decay=0),
    
    mask_type = "entmax",
    scheduler_params = dict(
        mode = "min", patience = 12, min_lr = 1e-5, factor = 0.9),
    scheduler_fn = ReduceLROnPlateau,
    seed = seed,
    verbose = 10
)

In [20]:

class LogitsLogLoss(Metric):
    """
    LogLoss with sigmoid applied
    """

    def __init__(self):
        self._name = "logits_ll"
        self._maximize = False

    def __call__(self, y_true, y_pred):
        """
        Compute LogLoss of predictions.

        Parameters
        ----------
        y_true: np.ndarray
            Target matrix or vector
        y_score: np.ndarray
            Score matrix or vector

        Returns
        -------
            float
            LogLoss of predictions vs targets.
        """
        logits = 1 / (1 + np.exp(-y_pred))
        aux = (1 - y_true) * np.log(1 - logits + 1e-15) + y_true * np.log(logits + 1e-15)
        return np.mean(-aux)

In [21]:
class LabelSmoothLoss(nn.Module):
    def __init__(self, smoothing=0.0,class_num=2):
        super(LabelSmoothLoss, self).__init__()
        assert 0 <= smoothing < 1
        self.smoothing = smoothing
        self.confidence = 1.0 - smoothing
        self.class_num = class_num
        
    def smooth_binary(self,input):
        one_confidence = torch.ones_like(input)*self.confidence
        one_smoothing = torch.ones_like(input)*self.smoothing/self.class_num
        if torch.cuda.is_available():
            one_confidence = one_confidence.cuda()
            one_smoothing = one_smoothing.cuda()
        
        m = torch.where(input >= 0.5, one_confidence, input)
        m = torch.where(input <= 0.5, one_smoothing, m)
        return m
    
    def forward(self, input, target):
        target = self.smooth_binary(target)
        return F.binary_cross_entropy_with_logits(input, target)

In [22]:
loss_with_label_smooth = LabelSmoothLoss(0.001,2)


In [23]:
scores_auc_all = []
test_cv_preds = []

oof_preds = []
oof_targets = []
scores = []
scores_auc = []

for fold_num in range(NB_SPLITS):
    print(b_,"FOLDS: ", r_, fold_num + 1)
    print(g_, '*' * 60, c_)
    test_fold = fold_num
    train_fold = [x for x in range(NB_SPLITS) if x!=fold_num]
    val_idx = targets['fold']==fold_num
    train_idx = targets['fold']!=fold_num
    
    X_train, y_train = train_df.values[train_idx, :], targets.values[train_idx, :-1]
    X_val, y_val = train_df.values[val_idx, :], targets.values[val_idx, :-1]
    ### Model ###
    model = TabNetRegressor(**tabnet_params)
        
    ### Fit ###
    # Another change to the original code
    # virtual_batch_size of 32 instead of 128
    model.fit(
        X_train = X_train,
        y_train = y_train,
        eval_set = [(X_val, y_val)],
        eval_name = ["val"],
        eval_metric = ["logits_ll"],
        max_epochs = MAX_EPOCH,
        patience = 30,
        batch_size = 512, 
        virtual_batch_size = 64,
        num_workers = 1,
        drop_last = False,
        # To use binary cross entropy because this is not a regression problem
        #loss_fn = F.binary_cross_entropy_with_logits
        loss_fn = loss_with_label_smooth
    )
    print(y_, '-' * 60)
    
    ### Predict on validation ###
    preds_val = model.predict(X_val)
    # Apply sigmoid to the predictions
    preds = 1 / (1 + np.exp(-preds_val))
    score = np.min(model.history["val_logits_ll"])
    
    ### Save OOF for CV ###
    oof_preds.append(preds_val)
    oof_targets.append(y_val)
    scores.append(score)
    
    ### Predict on test ###
    preds_test = model.predict(X_test)
    test_cv_preds.append(1 / (1 + np.exp(-preds_test)))

oof_preds_all = np.concatenate(oof_preds)
oof_targets_all = np.concatenate(oof_targets)
test_preds_all = np.stack(test_cv_preds)

 FOLDS:   1
 ************************************************************ 
Device used : cuda
epoch 0  | loss: 0.22326 | val_logits_ll: 0.02534 |  0:00:02s
epoch 10 | loss: 0.02115 | val_logits_ll: 0.01909 |  0:00:22s
epoch 20 | loss: 0.02068 | val_logits_ll: 0.01804 |  0:00:42s
epoch 30 | loss: 0.02044 | val_logits_ll: 0.01857 |  0:01:03s
epoch 40 | loss: 0.02022 | val_logits_ll: 0.01749 |  0:01:23s
epoch 50 | loss: 0.02032 | val_logits_ll: 0.0175  |  0:01:41s
epoch 60 | loss: 0.02018 | val_logits_ll: 0.0176  |  0:02:01s
epoch 70 | loss: 0.02003 | val_logits_ll: 0.01756 |  0:02:20s
epoch 80 | loss: 0.0199  | val_logits_ll: 0.01757 |  0:02:38s
epoch 90 | loss: 0.01993 | val_logits_ll: 0.01735 |  0:02:58s
epoch 100| loss: 0.01972 | val_logits_ll: 0.0175  |  0:03:17s
epoch 110| loss: 0.01974 | val_logits_ll: 0.01779 |  0:03:35s
epoch 120| loss: 0.01954 | val_logits_ll: 0.01735 |  0:03:55s
epoch 130| loss: 0.01939 | val_logits_ll: 0.01747 |  0:04:14s
epoch 140| loss: 0.01938 | val_logits_

In [24]:
np.save('oof.npy', oof_preds_all)

In [25]:
print('oof_preds_all.shape:',oof_preds_all.shape)
print('oof_targets_all.shape:',oof_targets_all.shape)
print('test_preds_all.shape:',test_preds_all.shape)

oof_preds_all.shape: (21948, 206)
oof_targets_all.shape: (21948, 206)
test_preds_all.shape: (7, 3624, 206)


In [26]:
aucs = []
for task_id in range(oof_preds_all.shape[1]):
    aucs.append(roc_auc_score(y_true = oof_targets_all[:, task_id],
                              y_score = oof_preds_all[:, task_id]
                             ))
print(f"{b_}Overall AUC: {r_}{np.mean(aucs)}")
print(f"{b_}Average CV: {r_}{np.mean(scores)}")

Overall AUC: 0.6286943190510096
Average CV: 0.017544881444833283


In [27]:
#submission
all_feat = [col for col in submission.columns if col not in ["sig_id"]]
# To obtain the same lenght of test_preds_all and submission
test = pd.read_csv(data_path + "test_features.csv")
sig_id = test[test["cp_type"] != "ctl_vehicle"].sig_id.reset_index(drop = True)
tmp = pd.DataFrame(test_preds_all.mean(axis = 0), columns = all_feat)
tmp["sig_id"] = sig_id

submission = pd.merge(test[["sig_id"]], tmp, on = "sig_id", how = "left")
submission.fillna(0, inplace = True)

#submission[all_feat] = tmp.mean(axis = 0)

# Set control to 0
#submission.loc[test["cp_type"] == 0, submission.columns[1:]] = 0
submission.to_csv("submission.csv", index = None)
submission.head()
print(f"{b_}submission.shape: {r_}{submission.shape}")

submission.shape: (3982, 207)
